In [1]:
# initialise the models
from robotsearch.robots import rct_robot
rct_clf = rct_robot.RCTRobot()

Using TensorFlow backend.
[DEBUG] robotsearch.ml.classifier 2018-05-10 13:30:42,421: Loading model /Users/iain/Code/robotsearch/robotsearch/data/rct/rct_svm_weights.npz...
[DEBUG] robotsearch.ml.classifier 2018-05-10 13:30:42,523: Model /Users/iain/Code/robotsearch/robotsearch/data/rct/rct_svm_weights.npz loaded
/Users/iain/anaconda3/lib/python3.5/site-packages/keras/engine/topology.py:1242: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  return cls(**config)


In [12]:
# an example abstract — genuine data from https://www.ncbi.nlm.nih.gov/pubmed/22269589
ti = """A randomized controlled trial of positive-affect induction to promote physical activity after percutaneous coronary intervention."""

ab = """BACKGROUND: Within 1 year after percutaneous coronary intervention, more than 20% of patients experience new adverse events. Physical activity confers a 25% reduction in mortality; however, physical activity is widely underused. Thus, there is a need for more powerful behavioral interventions to promote physical activity. Our objective was to motivate patients to achieve an increase in expenditure of 336 kcal/wk or more at 12 months as assessed by the Paffenbarger Physical Activity and Exercise Index.
METHODS: Two hundred forty-two patients were recruited immediately after percutaneous coronary intervention between October 2004 and October 2006. Patients were randomized to 1 of 2 groups. The patient education (PE) control group (n = 118) (1) received an educational workbook, (2) received a pedometer, and (3) set a behavioral contract for a physical activity goal. The positive-affect/self-affirmation (PA) intervention group (n = 124) received the 3 PE control components plus (1) a PA workbook chapter, (2) bimonthly induction of PA by telephone, and (3) small mailed gifts. All patients were contacted with standardized bimonthly telephone follow-up for 12 months.
RESULTS: Attrition was 4.5%, and 2.1% of patients died. Significantly more patients in the PA intervention group increased expenditure by 336 kcal/wk or more at 12 months, our main outcome, compared with the PE control group (54.9% vs 37.4%, P = .007). The PA intervention patients were 1.7 times more likely to reach the goal of a 336-kcal/wk or more increase by 12 months, controlling for demographic and psychosocial measures. In multivariate analysis, the PA intervention patients had nearly double the improvement in kilocalories per week at 12 months compared with the PE control patients (602 vs 328, P = .03).
CONCLUSION: Patients who receive PA intervention after percutaneous coronary intervention are able to achieve a sustained and clinically significant increase in physical activity by 12 months. Trial Registration clinicaltrials.gov Identifier: NCT00248846."""

ptyp = ["Journal Article", "Comment"]

to obtain predictions, call the `predict` function of the rct classifier as follows:

rct_clf.predict(input_list, filter_type, filter_class)

   input_list: an article (or list of articles) in the following format
          {"title": title, "abstract", abstract, "ptyp": ptyp, "use_ptyp": True/False}
          
          - ptyp is optional, and should be a list of *all* the listed ptyps in the MEDLINE entry
          - use_ptyp: whether to incorporate the ptyp into the model prediction.
              **NB THIS SHOULD BE `True` ONLY FOR ARTICLES WHERE MESH TAGGING IS COMPLETE**
              These are those with:
                  - `STAT -  MEDLINE` in PubMed RIS export
                  - `DB -    Ovid MEDLINE(R)` in Ovid RIS export
                  - `<MedlineCitation Status="MEDLINE" Owner="NLM">` in PubMed XML
              *all* others (including "PubMed-not-MEDLINE"/"OLDMEDLINE" and other close matches)
              should be used as use_ptyp=False
          
   filter_type: "sensitive", "balanced", or "precise"
       - different sensitivities as per our validation paper
       - as a rule of thumb, choose:
            'sensitive' for systematic reviews
            'balanced' for rapid reviews, or auto-generating trial registries
            'precise' for clinical question answering
   
   filter_class: "svm", "cnn", "svm_cnn" (the latter for an ensemble)
       - to help choose:
           - "svm" works a little better for sensitive strategies
           - "cnn" works a little better for precise strategies
           - "svm_cnn" is best for balanced strategies
       - all cutoffs can be used for all model types
       - cnns are slower, svms are near instant
     

In [26]:
# Calling the classifier for the example data
# params chosen here for making a registry of clinical trials
# (i.e. 'balanced', and 'svm_cnn')
pred = rct_clf.predict({"title": ti, "abstract": ab, "ptyp": ptyp, "use_ptyp": True}, filter_type="balanced", filter_class="svm_cnn")

	1 of 6
	2 of 6
	3 of 6
	4 of 6
	5 of 6
	6 of 6


In [27]:
print(pred)

[{'model': 'svm_cnn_ptyp', 'is_rct': True, 'ptyp_rct': 0, 'threshold_type': 'balanced', 'threshold_value': 1.8855674604380572, 'score': 4.211599629439782}]
